In [2]:
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np
import imutils
import time
import dlib
import cv2
import matplotlib.pyplot as plt
from keras.preprocessing.image import img_to_array
from keras.models import load_model


In [3]:
def eye_brow_distance(leye,reye):
    global points
    distq = dist.euclidean(leye,reye)
    points.append(int(distq))
    return distq

In [4]:
'''def rect_to_bb(rect):
	# take a bounding predicted by dlib and convert it
	# to the format (x, y, w, h) as we would normally do
	# with OpenCV
	x = rect.left()
	y = rect.top()
	w = rect.right() - x
	h = rect.bottom() - y

	# return a tuple of (x, y, w, h)
	return (x, y, w, h)'''

def emotion_finder(faces,frame):
    global emotion_classifier
    EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised","neutral"]
    x,y,w,h = face_utils.rect_to_bb(faces)
    frame = frame[y:y+h,x:x+w] #define frame from left to right and top to bottom
    # Extract region of interest(roi)
    roi = cv2.resize(frame,(64,64))
    roi = roi.astype("float") / 255.0
    #convert image pixels to numpy array
    roi = img_to_array(roi)
    roi = np.expand_dims(roi,axis=0)
    #CNN classification of predefine cnn clssifier into 7 classes
    preds = emotion_classifier.predict(roi)[0]
    emotion_probability = np.max(preds)
    label = EMOTIONS[preds.argmax()] #max probability class
    
    if label in ['scared','sad']:
        label = 'stressed'
    else:
        label = 'not stressed'
    return label


In [5]:
def normalize_values(points,disp):
    normalized_value = abs(disp - np.min(points))/abs(np.max(points) - np.min(points))
    stress_value = np.exp(-(normalized_value))
    print(stress_value)
    if stress_value>=75:
        return stress_value,"High Stress"
    else:
        return stress_value,"low_stress"

In [7]:
detector = dlib.get_frontal_face_detector() #return a detector we can use to retrieve the faces information
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") #retrive face features
# pretrained model for Emotion recognition
emotion_classifier = load_model("Emotion_detector.hdf5")
cap = cv2.VideoCapture(0)
points = []
while(True):
    _,frame = cap.read()
    frame = cv2.flip(frame,1) #horizontal flip
    frame = imutils.resize(frame, width=500,height=500) #resize frame
    
    #eye brow detect
    (lBegin, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eyebrow"]
    (rBegin, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eyebrow"]
    #print(lBegin,lEnd)
    #preprocessing the image(change orginonal image to gray)
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    detections = detector(gray)
    for detection in detections:
        emotion = emotion_finder(detection,gray)
        #draw emotion on image
        cv2.putText(frame, emotion, (10,10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        shape = predictor(frame,detection)
        
        '''def shape_to_np(shape, dtype="int"):
	      # initialize the list of (x, y)-coordinates
	      coords = np.zeros((shape.num_parts, 2), dtype=dtype)

	     # loop over all facial landmarks and convert them
	     # to a 2-tuple of (x, y)-coordinates
	       for i in range(0, shape.num_parts):
		    coords[i] = (shape.part(i).x, shape.part(i).y)

	       # return the list of (x, y)-coordinates
	       return coords'''
        
        shape = face_utils.shape_to_np(shape)
        #print(shape)
        leyebrow = shape[lBegin:lEnd]
        reyebrow = shape[rBegin:rEnd]
        #print(leyebrow)
        #convert shape into convexhull type
        reyebrowhull = cv2.convexHull(reyebrow)
        leyebrowhull = cv2.convexHull(leyebrow)
        #print(leyebrowhull)
        
        cv2.drawContours(frame, [reyebrowhull], -1, (0, 255, 0), 1) #Finding the contours gives us a list of boundary points around each blob
        cv2.drawContours(frame, [leyebrowhull], -1, (0, 255, 0), 1)

        distq = eye_brow_distance(leyebrow[-1],reyebrow[0])
        stress_value,stress_label = normalize_values(points,distq)
        cv2.putText(frame,"stress level:{}".format(str(int(stress_value*100))),(20,40),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
cv2.destroyAllWindows()
cap.release()
plt.plot(range(len(points)),points,'ro')
plt.title("Stress Levels")
plt.show()

[[181 324]
 [182 341]
 [183 357]
 [187 372]
 [194 386]
 [204 398]
 [216 409]
 [228 416]
 [242 418]
 [254 416]
 [263 407]
 [271 397]
 [279 385]
 [285 373]
 [289 359]
 [291 344]
 [292 329]
 [195 321]
 [204 314]
 [216 313]
 [228 316]
 [239 322]
 [254 324]
 [264 319]
 [274 317]
 [284 317]
 [290 323]
 [246 334]
 [246 346]
 [247 358]
 [247 370]
 [230 371]
 [237 374]
 [246 377]
 [254 374]
 [260 371]
 [206 334]
 [214 334]
 [221 334]
 [229 336]
 [221 338]
 [214 337]
 [259 337]
 [266 334]
 [273 335]
 [280 336]
 [273 339]
 [266 339]
 [220 384]
 [230 383]
 [239 383]
 [245 384]
 [250 383]
 [257 383]
 [265 384]
 [257 395]
 [250 400]
 [245 401]
 [238 401]
 [229 395]
 [224 385]
 [239 387]
 [245 387]
 [250 387]
 [261 385]
 [250 393]
 [245 394]
 [239 393]]
0.0


C:\Users\hario\anaconda3\envs\env_Sid\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  


[[159 281]
 [158 302]
 [158 322]
 [160 342]
 [165 363]
 [174 382]
 [187 398]
 [203 411]
 [223 417]
 [243 415]
 [260 403]
 [274 389]
 [286 371]
 [293 352]
 [297 331]
 [300 311]
 [300 290]
 [168 276]
 [178 268]
 [193 268]
 [207 272]
 [220 280]
 [242 282]
 [256 277]
 [271 274]
 [285 275]
 [295 285]
 [230 296]
 [229 311]
 [228 326]
 [228 341]
 [211 347]
 [218 351]
 [227 355]
 [236 352]
 [244 349]
 [182 294]
 [191 293]
 [200 293]
 [209 298]
 [199 300]
 [190 299]
 [252 301]
 [262 298]
 [271 298]
 [280 301]
 [271 306]
 [261 305]
 [201 373]
 [211 370]
 [220 368]
 [227 371]
 [234 369]
 [243 372]
 [254 378]
 [242 386]
 [233 388]
 [226 388]
 [218 386]
 [210 382]
 [206 373]
 [220 376]
 [227 378]
 [233 377]
 [248 377]
 [233 377]
 [226 377]
 [219 375]]
0.3631423798511207
[[162 280]
 [160 300]
 [160 320]
 [160 340]
 [164 360]
 [173 379]
 [185 394]
 [201 407]
 [221 412]
 [241 411]
 [259 401]
 [273 388]
 [285 371]
 [292 352]
 [297 333]
 [300 313]
 [300 293]
 [172 275]
 [182 266]
 [196 266]
 [210 271]
 

[[162 285]
 [159 304]
 [159 322]
 [159 341]
 [162 359]
 [170 376]
 [182 391]
 [197 402]
 [216 406]
 [235 405]
 [252 396]
 [266 384]
 [277 368]
 [285 351]
 [289 333]
 [292 315]
 [293 297]
 [169 277]
 [179 269]
 [192 269]
 [204 273]
 [216 280]
 [238 281]
 [250 277]
 [263 274]
 [276 275]
 [285 284]
 [226 294]
 [225 308]
 [224 320]
 [223 334]
 [208 341]
 [215 344]
 [222 347]
 [230 345]
 [238 343]
 [180 294]
 [188 290]
 [198 292]
 [207 298]
 [197 300]
 [187 299]
 [245 301]
 [254 295]
 [264 296]
 [271 300]
 [264 304]
 [254 304]
 [192 361]
 [204 359]
 [214 358]
 [221 360]
 [228 359]
 [238 362]
 [249 366]
 [237 374]
 [227 375]
 [219 375]
 [211 374]
 [202 371]
 [196 362]
 [213 365]
 [220 367]
 [228 366]
 [244 366]
 [227 365]
 [220 365]
 [213 364]]
0.49545856339350836
[[160 289]
 [158 307]
 [158 326]
 [160 344]
 [164 362]
 [172 380]
 [184 394]
 [199 405]
 [218 409]
 [237 407]
 [254 398]
 [268 386]
 [280 369]
 [286 351]
 [289 333]
 [291 315]
 [292 297]
 [169 278]
 [178 270]
 [191 270]
 [203 274]


[[163 283]
 [162 301]
 [162 318]
 [162 335]
 [166 351]
 [175 366]
 [189 378]
 [206 386]
 [224 389]
 [242 388]
 [259 381]
 [272 372]
 [282 358]
 [288 342]
 [290 325]
 [292 308]
 [292 291]
 [174 272]
 [184 264]
 [196 264]
 [209 267]
 [220 272]
 [243 273]
 [254 269]
 [267 266]
 [279 268]
 [286 277]
 [231 287]
 [231 299]
 [231 311]
 [231 324]
 [215 332]
 [222 334]
 [230 337]
 [238 335]
 [245 333]
 [186 291]
 [194 290]
 [202 290]
 [210 292]
 [202 295]
 [193 294]
 [250 293]
 [259 292]
 [267 293]
 [275 294]
 [267 297]
 [258 296]
 [200 354]
 [211 351]
 [221 349]
 [229 351]
 [236 349]
 [244 352]
 [254 357]
 [244 363]
 [235 365]
 [227 365]
 [219 364]
 [210 361]
 [205 354]
 [220 355]
 [228 357]
 [236 356]
 [250 357]
 [235 356]
 [228 356]
 [220 355]]
0.4483536831833937
[[164 283]
 [163 300]
 [163 316]
 [164 333]
 [169 349]
 [178 364]
 [191 375]
 [207 381]
 [226 383]
 [244 383]
 [261 377]
 [274 368]
 [283 354]
 [289 339]
 [291 322]
 [293 305]
 [293 288]
 [175 267]
 [185 260]
 [197 259]
 [209 262]
 

[[139 260]
 [136 275]
 [135 290]
 [135 306]
 [137 323]
 [143 337]
 [152 350]
 [166 358]
 [184 361]
 [203 360]
 [221 354]
 [235 343]
 [246 330]
 [252 314]
 [256 297]
 [257 281]
 [258 264]
 [142 242]
 [147 233]
 [157 232]
 [168 233]
 [177 236]
 [198 235]
 [210 231]
 [222 230]
 [234 234]
 [242 242]
 [187 250]
 [186 260]
 [184 269]
 [184 279]
 [171 287]
 [177 289]
 [185 292]
 [194 290]
 [202 288]
 [149 257]
 [156 255]
 [163 255]
 [171 257]
 [163 259]
 [156 259]
 [208 257]
 [216 255]
 [224 256]
 [230 259]
 [223 260]
 [216 259]
 [157 310]
 [167 303]
 [177 300]
 [186 302]
 [195 301]
 [207 304]
 [219 312]
 [207 325]
 [195 330]
 [185 330]
 [176 329]
 [166 323]
 [161 310]
 [177 306]
 [186 307]
 [195 306]
 [215 312]
 [195 319]
 [186 320]
 [176 318]]
0.5475072342357112
[[131 259]
 [129 274]
 [128 289]
 [128 306]
 [130 323]
 [136 338]
 [146 352]
 [160 362]
 [179 365]
 [199 363]
 [217 355]
 [231 344]
 [242 329]
 [249 312]
 [253 295]
 [254 278]
 [254 261]
 [133 242]
 [139 232]
 [149 230]
 [160 231]
 

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-6sxsq0tp\opencv\modules\imgproc\src\resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
